# Setup

In [ ]:
!pip3 install torch torchvision torchaudio
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)
from torchvision import transforms
import os
from pytorch_i3d import InceptionI3d
import moviepy.editor as mp

!pip3 install av
import av
print(av.__version__)

# !pip3 install opencv-python
!pip3 install opencv-contrib-python
import cv2
from denseflow import dense_flow

In [ ]:
!nvidia-smi
print(torch.cuda.is_available())

# Get Frames

## Resize videos while maintaining aspect ratio

In [7]:
def resize_video(old_video_path, new_video_name, new_video_root = "./demo_videos"):
    os.makedirs(os.path.join(new_video_root), exist_ok=True)
    clip = mp.VideoFileClip(old_video_path, audio=False)
    if(clip.w > clip.h):
        clip_resized = clip.resize(height=224) 
    else:
        clip_resized = clip.resize(width=224)
    clip_resized.write_videofile(os.path.join(new_video_root, new_video_name + ".mp4"))

In [8]:
resize_video("temp_videos/campfire.mp4", "campfire")

Moviepy - Building video ./demo_videos\campfire.mp4.
Moviepy - Writing video ./demo_videos\campfire.mp4



Moviepy - Done !
Moviepy - video ready ./demo_videos\campfire.mp4


## RGB

In [10]:
def video_to_rgb(video_path, video_name, new_rgb_root = "./demo_rgb"):
    os.makedirs(os.path.join(new_rgb_root, video_name), exist_ok=True)
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    count = 1
    while success:
        cv2.imwrite(os.path.join(new_rgb_root, video_name, video_name + "-%s.jpg" % str(count).zfill(6)), image)
        success, image = vidcap.read()
        count += 1

In [11]:
video_to_rgb("./demo_videos/campfire.mp4", "campfire")

## Flow


In [16]:
def video_to_flow(video_path, video_name, new_flow_root = "./demo_flow"):
    os.makedirs(os.path.join(new_flow_root, video_name), exist_ok=True)
    step = 1
    bound = 15
    dense_flow_args = [video_path, video_name, new_flow_root, step, bound]
    dense_flow(dense_flow_args)

In [ ]:
video_to_flow("./demo_videos/campfire.mp4", "campfire")

# Demo

In [ ]:
# print(torch.cuda.device(0))
# print(torch.cuda.get_device_name(0))
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"
device = torch.device(dev)
print(device)
rgb_i3d = InceptionI3d(400, in_channels=3)
rgb_i3d.replace_logits(8)
rgb_i3d.load_state_dict(torch.load("./newmodels_info/Training3/bnModel005000.pt", map_location=torch.device(device)))
rgb_i3d.eval()
flow_i3d = InceptionI3d(400, in_channels=2)
flow_i3d.replace_logits(8)
flow_i3d.load_state_dict(torch.load("./newmodels_info/Training1/flow_anamoly005000.pt", map_location=torch.device(device)))
flow_i3d.eval()

In [ ]:
from charades_dataset import load_rgb_frames, load_flow_frames, video_to_tensor
import videotransforms

cropTransform = transforms.Compose([videotransforms.CenterCrop(224)])

rgb_root = "./demo_rgb"
flow_root = "./demo_flow"
video = "campfire"

rgb_imgs = load_rgb_frames(rgb_root, video, 1, int(len(os.listdir(os.path.join(rgb_root, video)))))
rgb_imgs = cropTransform(rgb_imgs)
rgb_input = video_to_tensor(rgb_imgs)
rgb_input = rgb_input[None] # batch size of 1

flow_imgs = load_flow_frames(flow_root, video, 1, int(len(os.listdir(os.path.join(flow_root, video)))/2))
flow_imgs = cropTransform(flow_imgs)
flow_input = video_to_tensor(flow_imgs)
flow_input = flow_input[None] # batch size of 1

In [ ]:
# 0: Arson
# 1: Assault
# 2: Fighting
# 3: RoadAccidents
# 4: Shooting
# 5: Stealing
# 6: Vandalism
# 7: Normal
classes = ["Arson", "Assault", "Fighting", "RoadAccidents", "Shooting", "Stealing", "Vandalism", "Normal"]
# shape=(_BATCH_SIZE, _CHANNELS, _SAMPLE_VIDEO_FRAMES, _IMAGE_SIZE, _IMAGE_SIZE))
with torch.no_grad():
    rgb_prediction = rgb_i3d(rgb_input)
    flow_prediction = flow_i3d(flow_input)
prediction = (rgb_prediction + flow_prediction)/2.0

In [ ]:
averaged_prediction = torch.mean(prediction, dim=2)[0]
# print(averaged_prediction)
normalized_prediction = torch.softmax(averaged_prediction, dim=0)
# print([format(p, '.5f') for p in normalized_prediction])
final_class_index = torch.argmax(normalized_prediction)
# print(classes[final_class_index])

from prettytable import PrettyTable
t = PrettyTable(['Class', 'Probability'])
prediction_rows = [(i, normalized_prediction[i].item()) for i in range(8)]
from functools import cmp_to_key
prediction_rows = sorted(prediction_rows, reverse=True, key=cmp_to_key(lambda item1, item2: item1[1] - item2[1]))
for p in prediction_rows:
  t.add_row([classes[p[0]], format(p[1], '.5f')])
print(t)